In [31]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('white')

import sys
#sys.path.append("C:/Users/user/Desktop/ipynb/misshapen/")
sys.path.append("C:/Users/Yimeng/Documents/GitHub/")
from neurodsp import shape

import util
import Process
import imp
imp.reload(util)
imp.reload(Process)
from __future__ import print_function
from concurrent.futures import ThreadPoolExecutor

import os
import glob

In [3]:
data_paths, all_regions = util.load_paths()

In [4]:
# Help parse the large dataframe into small dataframes according to subject
def parseBehaviorDf(dfmove,keys):
    behavior_move_all={}
    for subj in keys:
        behavior_move_all[subj]=dfmove[dfmove['subject']==subj]
        behavior_move_all[subj] = behavior_move_all[subj].reset_index(drop=True)
    return behavior_move_all

def parseElectrode(paths):
    electrode=paths.split('\\')[1]
    return electrode[:-4]

def CalculateDFElectrode(elec_signals,f_range=(10,30),Fs=1000):
        findpt_kwargs = {'filter_fn':nonshape.bandpass_default,
                'filter_kwargs': {'w':3}}
        define_true_oscillating_periods_kwargs = {'ampdiff_th':.5, 'timediff_th':.6}
        df_P,df_T = shape.compute_shape_by_cycle(elec_signals, f_range, Fs,
                                          findpt_kwargs=findpt_kwargs,
                                          define_true_oscillating_periods_kwargs=define_true_oscillating_periods_kwargs)
       
        df_T_part = df_T[['rdsym_time','sample']]
        return df_T_part

In [5]:
dfmove = pd.read_csv('behavior_all.csv',index_col=0)
behavior_move_all=parseBehaviorDf(dfmove,all_regions.keys())

In [6]:
# Get the relation of electrodes and regions for each subject
electrode_df_all={}
for subj in all_regions.keys():
    elecs = []
    regions = []
    for region in data_paths[subj].keys():
        for paths in data_paths[subj][region]:
            elecs.append(parseElectrode(paths))
            regions.append(region)
    electrode_df_all[subj]=pd.DataFrame({'elec':pd.Series(elecs),'region':pd.Series(regions)})
    electrode_df_all[subj]['subject']=subj
    electrode_df_all[subj].to_csv('./elec-region_'+subj+'.csv')

In [7]:
def calculate_rdsym_average(behavior_move_all,data_paths,Fs=1000,f_range=(10,30),interval=500,move=True):
    tasks=[]
    e=ThreadPoolExecutor()
    for bmove_each in behavior_move_all.values():
        result = e.submit(calculate_rdsym_average_subj,bmove_each,data_paths,Fs,f_range,interval,move)
        tasks.append(result)
    for task in tasks:
        task.result()

In [32]:
util.calculate_rdsym_average(behavior_move_all,data_paths,interval=500,move=True)

C:\Users\Yimeng\Anaconda3\lib\site-packages\neurodsp\filt.py:213: UserWarning: Error when trying to estimate transition bandwidth. Visualize filter frequency response using the plot_frequency_response keyword argument to assure filter has appropriate response.
  warnings.warn('Error when trying to estimate transition bandwidth. Visualize filter frequency response using the plot_frequency_response keyword argument to assure filter has appropriate response.')
C:\Users\Yimeng\Anaconda3\lib\site-packages\neurodsp\shape\cyclepoints.py:127: RuntimeWarning: invalid value encountered in less
  pos = data < 0
C:\Users\Yimeng\Anaconda3\lib\site-packages\neurodsp\shape\cyclepoints.py:121: RuntimeWarning: invalid value encountered in greater
  pos = data > 0


In [35]:
def calculate_rdsym_average_subj(bmove_each,data_paths,Fs=1000,f_range=(10,30),interval=500,move=True):
    subj = bmove_each['subject'][0]
    move_start=bmove_each['move_start']
    move_end=bmove_each['move_end']
    dfneural_all_subj=pd.DataFrame({'start_time':move_start,'end_time':move_end})
    
    for region in data_paths[subj]:
        for path in data_paths[subj][region]:
            rdsym=[]
            electrode = parseElectrode(path)
            signal = np.load(path) # voltage series
        
            signal = Process.preprocess_filters(signal,Fs)
            df_electrode = shape.features_by_cycle(signal, Fs, f_range, center_extrema='T' )
            df_electrode['subject']=subj
            df_electrode.dropna()
            for index in range(len(move_start)):
                if move:
                    trial = df_electrode[(df_electrode['sample_trough']<=move_end[index]) & (df_electrode['sample_trough']>=move_start[index])]
                    avg=util.calculate_average(trial,'time_rdsym')
                else:
                     if index != len(move_start)-1: 
                        if move_start[index+1]-move_end[index]<2*interval:
                            avg=np.nan
                        else:
                            middle = (move_end[index]+move_start[index+1])/2
                            trial= df_electrode[(df_electrode['sample_trough']>=middle-interval) & (df_electrode['sample_trough']<=middle+interval)]
                            avg=util.calculate_average(trial,'time_rdsym')
                rdsym.append(avg)
            rdsym_series=pd.Series(rdsym)
    
            dfneural_all_subj['rdsym_elec'+electrode]=rdsym_series.values
    dfneural_all_subj.reindex_axis(sorted(dfneural_all_subj.columns), axis=1)
    if move:
        dfneural_all_subj.to_csv('./neural_move_'+subj+'.csv')
    else:
        dfneural_all_subj.to_csv('./neural_nomove_'+subj+'.csv')

In [33]:
util.calculate_rdsym_average(behavior_move_all,data_paths,interval=500,move=False)

C:\Users\Yimeng\Anaconda3\lib\site-packages\neurodsp\filt.py:213: UserWarning: Error when trying to estimate transition bandwidth. Visualize filter frequency response using the plot_frequency_response keyword argument to assure filter has appropriate response.
  warnings.warn('Error when trying to estimate transition bandwidth. Visualize filter frequency response using the plot_frequency_response keyword argument to assure filter has appropriate response.')
C:\Users\Yimeng\Anaconda3\lib\site-packages\neurodsp\shape\cyclepoints.py:127: RuntimeWarning: invalid value encountered in less
  pos = data < 0
C:\Users\Yimeng\Anaconda3\lib\site-packages\neurodsp\shape\cyclepoints.py:121: RuntimeWarning: invalid value encountered in greater
  pos = data > 0
